# sequence_encoder

level追加

In [1]:
import os
import sys
import traceback
import gc
import random
import pickle
import pathlib
import subprocess
from dataclasses import dataclass

import pandas as pd
import polars as pl
import numpy as np
from sklearn.metrics import f1_score
from sklearn.model_selection import GroupKFold
import lightgbm as lgb

import matplotlib.pyplot as plt
import seaborn as sns
from tqdm import tqdm


/opt/conda/lib/python3.10/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.23.5
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


In [2]:
@dataclass
class Cfg:
    input_dir = "/mnt/predict-student-performance-from-game-play/input/"
    output_dir = "/mnt/predict-student-performance-from-game-play/output/"
    prep_dir = "/mnt/predict-student-performance-from-game-play/prep/"
cfg = Cfg()

In [3]:
level_groups = ['0-4', '5-12', '13-22']

In [31]:
train = pd.read_csv(cfg.input_dir + "train.csv")

In [32]:
room_fqids = train["room_fqid"].unique().tolist()
room_fqid_encoder = {room_fqid: str(i) for i,room_fqid in enumerate(room_fqids)}
room_fqid_decoder = {str(i): room_fqid for i,room_fqid in enumerate(room_fqids)}

In [33]:
sequence_encoders = {}
for group in level_groups:
    df = train[train["level_group"]==group]
    
    break

    room_fqid_sequences = list(df["room_fqid_encode"].unique())
    room_fqid_sequence_encoder = {room_fqid_seq: i for i,room_fqid_seq in enumerate(room_fqid_sequences)}
    sequence_encoders[group] = {}
    sequence_encoders[group]["room_fqid"] = room_fqid_sequence_encoder

In [34]:
df["room_fqid"] = df["room_fqid"].map(room_fqid_encoder)

/tmp/ipykernel_3334/588865456.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df["room_fqid"] = df["room_fqid"].map(room_fqid_encoder)


In [38]:
df = df[df["room_fqid"]!=df["room_fqid"].shift(1)].copy()

In [39]:
agg_df = df.groupby("session_id")["room_fqid"].apply(list).reset_index()

In [41]:
from gensim.models import Word2Vec

In [42]:
sentences = agg_df["room_fqid"].tolist()

In [55]:
model = Word2Vec(sentences=sentences, seed=42, vector_size=20)

In [56]:
model.train(sentences, total_examples=len(sentences), epochs=model.epochs)

(97787, 1104685)

In [58]:
model.wv["1"].shape

(20,)

In [70]:
len(sentences)

23562

In [65]:
for s in sentences:
    
        

0
1
2
3
2
4
2
1
0
1
2
5
6


In [78]:
np.mean(([model.wv[w] for w in s]), axis=0)

array([ 0.3662354 , -0.29645354,  0.08815043, -0.2601917 ,  0.14198613,
        0.19885486,  0.27727878, -0.01053757, -0.04869536,  0.1437256 ,
       -0.18036431,  0.2527313 ,  0.22556281, -0.08455282,  0.1077632 ,
        0.05164345, -0.4484348 , -0.20134228, -0.07870914, -0.34894282],
      dtype=float32)

In [43]:
with open(cfg.prep_dir + "room_fqid_encoder.pkl", "wb") as f:
    pickle.dump(room_fqid_encoder, f)

with open(cfg.prep_dir + "sequence_encoders.pkl", "wb") as f:
    pickle.dump(sequence_encoders, f)

In [42]:
#df.groupby("session_id").agg(pl.col("room_fqid_encode").filter(pl.col("room_fqid_encode")!=pl.col("room_fqid_encode").shift(1)).str.concat("").map_dict(room_fqid_sequence_encoder))